In [7]:
import pandas as pd
import numpy as np
import pprint

#not needed right now (maybe ever?) doing one-hot encoding next cell
#might have to use if we are doing the neutral thing
s = """
#encodes a column of parties into -1, 0, or 1
def encode_party(party_series):
    types = {"Democrat": 0, "Neutral": 0, "Republican": 0}
    output_array = []
    for party in party_series:
        if party == "Democrat":
            types["Democrat"] += 1
        elif party == "Neutral":
            types["Neutral"] += 1
        elif party == "Republican":
            types["Republican"] += 1
            
    resulting_encoding = []

    for type_ in types.keys():
        resulting_encoding.append(type_)
    resulting_encoding.sort()
    return np.asarray(resulting_encoding)
"""

In [8]:
#Hydrating data
tweets = pd.read_csv("data/ExtractedTweets.csv")

#Preprocessing data

#one-hot encoding republican and democrat
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(dtype=np.int, sparse=True, categories='auto')
parties = pd.DataFrame(onehot.fit_transform(tweets[["Party"]])\
                            .toarray(),
                            columns=["Democrat", "Republican"]) #todo add neutral column & training data
parties["Handle"] = tweets.Handle
tweets = tweets.drop(["Party"], axis=1)
encoded_data = pd.merge(parties, tweets)

#Concatinates all tweets in a user's recent tweets, and tokenizes the result
from nltk.tokenize import TweetTokenizer
def concatinate_tweets(all_tweets):
    document = " "
    for tweet in all_tweets:
        for single_tweet in tweet.values[0:200]: 
            document += single_tweet + " \n "
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(document)

#dictionary containing the user's handle and a tokenized representation of 200 of their tweets
handles_and_tweets = {}
for handle in encoded_data.groupby(["Handle"])["Tweet"]:
    handles_and_tweets[handle[0]] = concatinate_tweets(handle[1:])

In [9]:
#tf-idf and google word2vec
#TODO: vectorize the tweets (for each value in handles_and_tweets.items())

# TODO : tf-idf

# W2V encoding
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('train/GoogleNews-vectors-negative300.bin.gz', binary=True)
# TODO : use what was done in hw2


ModuleNotFoundError: No module named 'gensim'

In [4]:
#Organizing the data for training here
df = pd.DataFrame.from_dict(handles_and_tweets, orient='index', columns=['Tweets'])
df = df.reset_index()
df.rename(columns={'index':'Handle'}, inplace=True)
finished_product = pd.merge(parties, df, how='right')
finished_product = finished_product.set_index('Handle')
finished_product.drop_duplicates()

ValueError: 1 columns passed, passed data had 4843 columns

In [10]:
#Splitting up the data into train and test
from sklearn.model_selection import train_test_split
                            #Insert formatted 2d array here vvv
X_train, X_test, Y_train, Y_test = train_test_split(finished_product, test_size = 0.2)

ValueError: not enough values to unpack (expected 4, got 2)

In [6]:
#Training on model - MLP classifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
#Grid search CV
parameters = {'solver': ['lbfgs'],
              'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ],
              'alpha': 10.0 ** -np.arange(1, 10),
              'hidden_layer_sizes':np.arange(10, 15)}

tuned_MLP_classifier = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
tuned_MLP_classifier.fit(X_train, Y_train)
print(tuned_MLP_classifier.score(X_train, Y_train))
print(tuned_MLP_classifier.best_params_)

NameError: name 'np' is not defined

In [ ]:
#Evaluation for MLP Classifier

y_predictions = tuned_MLP_classifier.predict(x_test)
print(confusion_matrix(y_test.astype(int), y_predictions.astype(int)))
print(accuracy_score(y_test.astype(int), y_predictions.astype(int)))


In [ ]:
#Training on model - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

#Grid search for hyperparameter tuning (for random forest)
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint.pprint(random_grid)
tuned_RF_classifier = RandomizedSearchCV(estimator = model, param_distributions = random_grid,
                                  n_iter = 100, cv = 3, verbose=2, random_state=42,
                                  n_jobs = -1)# Fit the random search model
tuned_RF_classifier.fit(x_train, y_train)

In [ ]:
#Evaluation for RandomForest Classifier

#Evaluation:
print(f'Training Score: {tuned_RF_classifier.score(x_train, y_train)} '
      + f'\n Validation Score: {tuned_RF_classifier.score(x_test, y_test)}')